In [81]:
import pandas as pd 
import pandas_ta as ta 
import yfinance as yf 
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio
from itertools import pairwise
from smartmoneyconcepts import smc

In [2]:
def cleaning(df):
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    df.columns = [name[0].lower() for name in df.columns]  # Convert column names to lowercase
    df.index = df.index.rename('date')  # Rename index to 'date'
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d %H:%M:%S')  # Format index as datetime string
    df = df.reset_index(drop=False)  # Reset index and assign back to df
    return df

In [136]:
def resampling(df, interval='1D'):
    df.index = pd.DatetimeIndex(df['date'])
    
    df = df.resample(interval).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
    return df

In [39]:
gold_5min = cleaning(yf.download('GC=F', period='max', end=datetime.now(), interval='5m'))

[*********************100%***********************]  1 of 1 completed


In [40]:
gold_5min

,date,close,high,low,open,volume
0,2024-12-02 11:20:00,2664.399902,2666.800049,2662.800049,2665.399902,0
1,2024-12-02 11:25:00,2662.500000,2665.000000,2662.199951,2664.399902,525
2,2024-12-02 11:30:00,2666.399902,2666.399902,2659.899902,2662.500000,1865
3,2024-12-02 11:35:00,2666.000000,2669.000000,2665.399902,2666.300049,1802
4,2024-12-02 11:40:00,2668.399902,2668.699951,2665.600098,2666.100098,1007
...,...,...,...,...,...,...
11100,2025-01-31 02:50:00,2849.300049,2849.500000,2848.600098,2848.600098,136
11101,2025-01-31 02:55:00,2848.399902,2849.500000,2848.399902,2849.300049,79
11102,2025-01-31 03:00:00,2848.600098,2849.000000,2847.899902,2848.399902,161
11103,2025-01-31 03:05:00,2850.300049,2851.199951,2848.199951,2848.600098,376


In [64]:
gold_daily = resampling(gold_5min)
gold_daily

,open,high,low,close
date,,,,
2024-12-02,2665.399902,2675.000000,2635.699951,2664.699951
2024-12-03,2664.899902,2678.399902,2636.000000,2664.699951
2024-12-04,2664.699951,2682.000000,2637.000000,2673.899902
2024-12-05,2674.000000,2680.000000,2625.699951,2656.699951
2024-12-06,2656.600098,2667.899902,2617.500000,2654.899902
...,...,...,...,...
2025-01-27,2772.800049,2777.800049,2732.000000,2745.399902
2025-01-28,2745.600098,2774.000000,2737.399902,2769.800049
2025-01-29,2769.800049,2778.100098,2761.000000,2773.600098


In [137]:
def CRT(df):
    fvg = smc.fvg(df)
    fvg.index = df.index
    df_fvg = pd.concat([df, fvg], axis=1)

    df_daily = resampling(df, interval='1D')



    
    # Bullish condition
    cU = (df_daily['close'] > df_daily['open']) & (df_daily['close'].shift(1) < df_daily['open'].shift(1))
    
    # Bearish condition
    cd = (df_daily['close'] < df_daily['open']) & (df_daily['close'].shift(1) > df_daily['open'].shift(1))

    # New High
    cu1 = (df_daily['high'] > df_daily['high'].shift(1)) & (df_daily['low'] < df_daily['low'].shift(1))
    
    # New Low
    cd1 = (df_daily['low'] > df_daily['low'].shift(1)) & (df_daily['high'] < df_daily['high'].shift(1))

    
    return df_daily.loc[(cU | cd) & (cu1 | cd1)], df_fvg, df

In [138]:
CRT(gold_5min)[0]

,open,high,low,close
date,,,,
2024-12-09 00:00:00,2662.199951,2700.000000,2630.800049,2683.000000
2024-12-12 00:00:00,2753.899902,2761.300049,2677.399902,2704.899902
2024-12-16 00:00:00,2669.199951,2683.399902,2651.000000,2668.399902
2024-12-19 00:00:00,2601.699951,2640.899902,2601.000000,2608.800049
2024-12-23 00:00:00,2641.500000,2646.800049,2623.199951,2629.899902
2024-12-24 00:00:00,2629.800049,2635.800049,2624.199951,2634.199951
2024-12-30 00:00:00,2638.899902,2640.699951,2608.399902,2620.199951
2025-01-06 00:00:00,2653.600098,2663.800049,2624.600098,2645.100098
2025-01-13 00:00:00,2721.000000,2723.800049,2675.199951,2686.000000


In [133]:
CRT(gold_5min)[1]

,date,close,high,low,open,volume,FVG,Top,Bottom,MitigatedIndex
date,,,,,,,,,,
2024-12-02 11:20:00,2024-12-02 11:20:00,2664.399902,2666.800049,2662.800049,2665.399902,0,NaN,NaN,NaN,NaN
2024-12-02 11:25:00,2024-12-02 11:25:00,2662.500000,2665.000000,2662.199951,2664.399902,525,NaN,NaN,NaN,NaN
2024-12-02 11:30:00,2024-12-02 11:30:00,2666.399902,2666.399902,2659.899902,2662.500000,1865,1.0,2665.399902,2665.0,6.0
2024-12-02 11:35:00,2024-12-02 11:35:00,2666.000000,2669.000000,2665.399902,2666.300049,1802,NaN,NaN,NaN,NaN
2024-12-02 11:40:00,2024-12-02 11:40:00,2668.399902,2668.699951,2665.600098,2666.100098,1007,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-01-31 02:50:00,2025-01-31 02:50:00,2849.300049,2849.500000,2848.600098,2848.600098,136,NaN,NaN,NaN,NaN
2025-01-31 02:55:00,2025-01-31 02:55:00,2848.399902,2849.500000,2848.399902,2849.300049,79,NaN,NaN,NaN,NaN
2025-01-31 03:00:00,2025-01-31 03:00:00,2848.600098,2849.000000,2847.899902,2848.399902,161,NaN,NaN,NaN,NaN


In [104]:
smc.fvg(gold_5min, join_consecutive=True)

,FVG,Top,Bottom,MitigatedIndex
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,1.0,2665.399902,2665.0,6.0
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
11100,NaN,NaN,NaN,NaN
11101,NaN,NaN,NaN,NaN
11102,NaN,NaN,NaN,NaN
11103,NaN,NaN,NaN,NaN


In [83]:
gold_5min

,date,close,high,low,open,volume
date,,,,,,
2024-12-02 11:20:00,2024-12-02 11:20:00,2664.399902,2666.800049,2662.800049,2665.399902,0
2024-12-02 11:25:00,2024-12-02 11:25:00,2662.500000,2665.000000,2662.199951,2664.399902,525
2024-12-02 11:30:00,2024-12-02 11:30:00,2666.399902,2666.399902,2659.899902,2662.500000,1865
2024-12-02 11:35:00,2024-12-02 11:35:00,2666.000000,2669.000000,2665.399902,2666.300049,1802
2024-12-02 11:40:00,2024-12-02 11:40:00,2668.399902,2668.699951,2665.600098,2666.100098,1007
...,...,...,...,...,...,...
2025-01-31 02:50:00,2025-01-31 02:50:00,2849.300049,2849.500000,2848.600098,2848.600098,136
2025-01-31 02:55:00,2025-01-31 02:55:00,2848.399902,2849.500000,2848.399902,2849.300049,79
2025-01-31 03:00:00,2025-01-31 03:00:00,2848.600098,2849.000000,2847.899902,2848.399902,161


In [142]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio
from itertools import pairwise
from smartmoneyconcepts import smc

# Step 1: Fetch historical data using yfinance (or MetaTrader5 if you're connected)
# Example using yfinance
symbol = "EURUSD=X"  # Example: EUR/USD for Forex
start_date = datetime.now() - timedelta(days=90)  # Last 90 days of data
end_date = datetime.now()

# Fetching data from Yahoo Finance
df = yf.download(symbol, start=start_date, end=end_date, interval='1h')  # Hourly data

# Step 2: Calculate Candlestick Range (Difference between High and Low)
df['candle_range'] = df['High'] - df['Low']

# Rename columns to lowercase
df.columns = df.columns.str.lower()

# Step 3: Smart Money Concepts - Apply Fair Value Gap (FVG)
fvg = smc.fvg(df)  # Now it should work correctly

# Step 4: Technical Indicators - Add moving averages (e.g., SMA, EMA)
df['SMA_20'] = ta.sma(df['close'], 20)
df['EMA_50'] = ta.ema(df['close'], 50)

# Step 5: Generate Signals based on Candle Range and Moving Averages
df['buy_signal'] = (df['candle_range'] > df['candle_range'].rolling(10).mean()) & (df['close'] > df['EMA_50'])
df['sell_signal'] = (df['candle_range'] > df['candle_range'].rolling(10).mean()) & (df['close'] < df['EMA_50'])

# Step 6: Plotting the Data using Plotly
fig = go.Figure()

# Candlestick chart
fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlesticks'))

# Plot Buy/Sell signals
fig.add_trace(go.Scatter(x=df.index[df['buy_signal']], y=df['close'][df['buy_signal']], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='green', size=10)))
fig.add_trace(go.Scatter(x=df.index[df['sell_signal']], y=df['close'][df['sell_signal']], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='red', size=10)))

# Add FVGs (if applicable, depending on how you want to highlight)
# For now, this is a placeholder since the function smc.fvg() needs to return specific columns
# fig.add_trace(go.Scatter(x=df.index[fvg['gap_start']], y=df['close'][fvg['gap_start']], mode='lines', name='FVG Start'))

# Update layout
fig.update_layout(title=f"{symbol} Candlestick Range Strategy", xaxis_title="Date", yaxis_title="Price", xaxis_rangeslider_visible=False)
pio.show(fig, renderer='browser')


[*********************100%***********************]  1 of 1 completed


AttributeError: Can only use .str accessor with Index, not MultiIndex